<a href="https://colab.research.google.com/github/abhishek-pes/DA-Project-Amazon-product-listing/blob/main/AmazonNlpTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TEAM DATA 404
## Amazon product classification

Test playground No EDA

In [37]:
import pandas as pd
import numpy as np
from datetime import datetime

In [38]:
url = 'https://raw.githubusercontent.com/abhishek-pes/DA-Project-Amazon-product-listing/main/test/cleaned_data.csv?token=AM4ZPXAW7VJBTGN6XOEFIGLBW4YHO'
amzn = pd.read_csv(url, error_bad_lines=False)
amzn.head()

,Unnamed: 0,Uniq Id,Crawl Timestamp,Category,Product Title,Product Description,Brand,Pack Size Or Quantity,Mrp,Price,Offers,Combo Offers,Stock Availibility
0,0,eb49cc038190f6f03c272f79fbbce894,2019-10-30 11:38:11+00:00,Skin Care,Lee posh Lactic Acid 60% Anti ageing Pigmenta...,PROFESSIONAL GRADE Face Peel: this peel stimul...,Lee Posh,-,2000.0,799.0,60.050000,0,1
1,1,1657cc30c438affede6a5060d6847363,2019-10-31 15:46:54+00:00,Skin Care,Branded SLB Works New 1.5mm Titanium 1200 nee...,Item name: 1.5mm titanium 1200 needles microne...,SLB Works,-,2040.0,2040.0,0.000000,0,1
2,2,41654633cce38c8650690f6dbac01fd3,2019-10-30 09:53:23+00:00,Skin Care,Generic 1 Pc brand snail eye cream remove dar...,"Use: eye, item type: cream, net wt: 20g, gzzz:...",Generic,-,1824.0,1042.0,42.872807,0,1
3,3,08b1bd85c3efc2d7aa556fd79b073382,2019-10-29 16:16:52+00:00,Skin Care,Generic Anti Snoring Snore Stopper Sleep Apne...,Prevent the tongue from dropping backward or b...,Generic,-,2185.0,1399.0,35.972540,0,1
4,4,f89b246d4e27c11623dbc7742523f319,2019-10-30 19:14:20+00:00,Skin Care,"Shikai Borage Dry Skin Therapy Foot Cream, 4....","Package Quantity:3 Contains borage oil, clinic...",ShiKai,354 g,5344.0,5344.0,0.000000,0,1


In [39]:
# converting to lower case
amzn['Product Description'] =  amzn['Product Description'].str.lower()

In [40]:
# removing punctuations and numnerical values
import re,string
amzn['Product Description'] = amzn['Product Description'].str.replace('[^\w\s]','')
amzn['Product Description'] = amzn['Product Description'].str.replace('\d+', '')

In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
amzn['Product Description'] = amzn['Product Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [43]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
#defining function for tokenization
from nltk.tokenize import sent_tokenize, word_tokenize
def tokenization(text):
    tokens = sent_tokenize(text)
    return tokens
#applying function to the column
amzn['Product Description']= amzn['Product Description'].apply(lambda x: tokenization(x))

In [45]:
amzn['Product Description']

0        [professional grade face peel peel stimulates ...
1        [item name mm titanium needles microneedles me...
2        [use eye item type cream net wt g gzzz ygzwbz ...
3        [prevent tongue dropping backward block airway...
4        [package quantity contains borage oil clinical...
                               ...                        
28005    [needful sky drop washing liquid laundry deter...
28006    [tides new ultra matic laundry detergent powde...
28007    [new ariel matic liquid detergent gives tough ...
28008    [needful cleans softens freshens protects fabr...
28009    [item package quantity zout triple enzyme form...
Name: Product Description, Length: 28010, dtype: object

In [46]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

In [47]:
#defining a function for stemming
def stemming(text):
  stem_text = [porter_stemmer.stem(word) for word in text]
  return stem_text
amzn['Product Description']=amzn['Product Description'].apply(lambda x: stemming(x))

In [48]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

In [49]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text
amzn['Product Description']=amzn['Product Description'].apply(lambda x:lemmatizer(x))

In [51]:
# Need to convert to vector form using tfidf, bag of words or count vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
amzn['test_text'] = amzn['Product Description'].apply(lambda x: ' '.join(x))

In [52]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(amzn['test_text'])

## Model Building

### Splitting dataset

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [54]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,r2_score,f1_score
from math import sqrt

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, amzn['Category'], test_size=0.30, random_state = 50)    
clf = SVC(kernel='linear').fit(X_train, y_train)

### Support vector

In [56]:
predictions = clf.predict(X_test)
from sklearn.metrics import classification_report
print (classification_report(y_test, predictions))

                         precision    recall  f1-score   support

          Bath & Shower       0.70      0.28      0.40       647
  Detergents & Dishwash       0.96      0.51      0.67        53
              Fragrance       0.85      0.81      0.83       582
Grocery & Gourmet Foods       0.98      0.97      0.98      2195
              Hair Care       0.91      0.79      0.85       692
              Skin Care       0.85      0.96      0.90      4234

               accuracy                           0.88      8403
              macro avg       0.87      0.72      0.77      8403
           weighted avg       0.88      0.88      0.87      8403



In [57]:
print("accuracy on test:",accuracy_score(y_test,predictions))
svc_score = accuracy_score(y_test,predictions)

accuracy on test: 0.88230393906938


### Multinomial Naive bayes

In [58]:
from sklearn.naive_bayes import MultinomialNB
mNb = MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [59]:
mNb.fit(X_train,y_train)

MultinomialNB(alpha=0.1)

In [60]:
predictions_mNb = mNb.predict(X_test)
print (classification_report(y_test, predictions_mNb,labels=np.unique(predictions_mNb)))

                         precision    recall  f1-score   support

          Bath & Shower       0.59      0.32      0.42       647
  Detergents & Dishwash       1.00      0.25      0.39        53
              Fragrance       0.84      0.85      0.85       582
Grocery & Gourmet Foods       0.98      0.98      0.98      2195
              Hair Care       0.91      0.77      0.83       692
              Skin Care       0.86      0.94      0.90      4234

               accuracy                           0.88      8403
              macro avg       0.86      0.68      0.73      8403
           weighted avg       0.87      0.88      0.87      8403



In [61]:
print("accuracy:",accuracy_score(y_test,predictions_mNb))
mnb_score = accuracy_score(y_test,predictions_mNb)

accuracy: 0.8790908009044389


### Random forest classifier

In [62]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200,max_depth=8, random_state=0,criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=200,
                       random_state=0)

In [63]:
predictions_rfc = rfc.predict(X_test)
print (classification_report(y_test, predictions_rfc,labels=np.unique(predictions_rfc)))

                         precision    recall  f1-score   support

              Fragrance       1.00      0.10      0.19       582
Grocery & Gourmet Foods       1.00      0.31      0.47      2195
              Skin Care       0.55      1.00      0.71      4234

              micro avg       0.59      0.71      0.64      7011
              macro avg       0.85      0.47      0.46      7011
           weighted avg       0.73      0.71      0.59      7011



In [64]:
print("accuracy:",accuracy_score(y_test,predictions_rfc))
rfc_score = accuracy_score(y_test,predictions_rfc)

accuracy: 0.5915744377008212


### Gradient boosting

In [65]:
from xgboost import XGBClassifier

In [66]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

In [67]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred,labels=np.unique(y_pred)))
# evaluate predictions
xgb_score = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (xgb_score * 100.0))

                         precision    recall  f1-score   support

          Bath & Shower       0.63      0.32      0.42       647
  Detergents & Dishwash       1.00      0.47      0.64        53
              Fragrance       0.83      0.77      0.80       582
Grocery & Gourmet Foods       0.97      0.82      0.89      2195
              Hair Care       0.87      0.79      0.83       692
              Skin Care       0.79      0.94      0.86      4234

               accuracy                           0.83      8403
              macro avg       0.85      0.68      0.74      8403
           weighted avg       0.84      0.83      0.82      8403

Accuracy: 83.30%


### Perceptron model

In [68]:
from sklearn.linear_model import Perceptron
perc = Perceptron(penalty=None, alpha=0.5, fit_intercept=True, shuffle=False, verbose=0, eta0=1.0, n_jobs=1, random_state=0, class_weight=None, warm_start=False)
perc.fit(X_train,y_train)

Perceptron(alpha=0.5, n_jobs=1, shuffle=False)

In [69]:
predictions_perc = perc.predict(X_test)
print (classification_report(y_test, predictions_perc,labels=np.unique(predictions_perc)))

                         precision    recall  f1-score   support

          Bath & Shower       0.54      0.43      0.48       647
  Detergents & Dishwash       0.59      0.89      0.71        53
              Fragrance       0.81      0.89      0.85       582
Grocery & Gourmet Foods       0.95      0.98      0.96      2195
              Hair Care       0.82      0.87      0.84       692
              Skin Care       0.90      0.89      0.90      4234

               accuracy                           0.88      8403
              macro avg       0.77      0.82      0.79      8403
           weighted avg       0.87      0.88      0.87      8403



In [70]:
print("accuracy:",accuracy_score(y_test,predictions_perc))
percp_score = accuracy_score(y_test,predictions_perc)

accuracy: 0.876710698560038
